隨機抽 Anterior_crop Posterior_crop 20000 變 train-dataset 並且要定位清楚   Anterior Posterior 的照片

In [4]:
#import os
#import random
#import shutil
#from pathlib import Path

# 設定路徑
#anterior_path = Path("Anterior_crop")
#posterior_path = Path("Posterior_crop")
#train_path = Path("cut/train")

# 建立資料夾
#(train_path / "Anterior").mkdir(parents=True, exist_ok=True)
#(train_path / "Posterior").mkdir(parents=True, exist_ok=True)

# 遞迴找圖片檔案
#anterior_images_all = list(anterior_path.rglob("*.jpg"))
#posterior_images_all = list(posterior_path.rglob("*.jpg"))

# 隨機抽樣
#anterior_count = min(10000, len(anterior_images_all))
#posterior_count = min(10000, len(posterior_images_all))

#anterior_images = random.sample(anterior_images_all, anterior_count)
#posterior_images = random.sample(posterior_images_all, posterior_count)

# 複製圖片
#for img in anterior_images:
#    shutil.copy(img, train_path / "Anterior" / img.name)

#for img in posterior_images:
#    shutil.copy(img, train_path / "Posterior" / img.name)


資料集定義

In [5]:
import os
import csv
from pathlib import Path

# 資料夾路徑
anterior_path = Path(r"C:\Users\f9104\Desktop\cut\cut\train\Anterior")
posterior_path = Path(r"C:\Users\f9104\Desktop\cut\cut\train\Posterior")

# 定義支援的圖片副檔名
extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']

# 收集所有圖片檔案
anterior_images = [f for f in anterior_path.iterdir() if f.suffix.lower() in extensions]
posterior_images = [f for f in posterior_path.iterdir() if f.suffix.lower() in extensions]

# 寫入 CSV（標註：Anterior=0, Posterior=1）
with open("image_labels.csv", "w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['filepath', 'label'])  # 欄位名稱

    for img in anterior_images:
        writer.writerow([str(img), 0])
    for img in posterior_images:
        writer.writerow([str(img), 1])

print("✅ image_labels.csv 已成功建立！")


✅ image_labels.csv 已成功建立！


In [6]:
# 資料夾路徑
anterior_path = r"C:\Users\f9104\Desktop\cut\cut\train\Anterior"
posterior_path = r"C:\Users\f9104\Desktop\cut\cut\train\Posterior"


In [7]:
import random  
image_file_paths_with_labels = []

for folder in [anterior_path, posterior_path]:
    label = os.path.basename(folder)  # 'Anterior' 或 'Posterior'
    for fname in os.listdir(folder):
        if fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            full_path = os.path.join(folder, fname)
            image_file_paths_with_labels.append((full_path, label))

# 隨機選 10 張圖片
random_samples = random.sample(image_file_paths_with_labels, 10)

# 印出圖檔與標籤
for img_path, label in random_samples:
    print(f"{label}: {img_path}")


Anterior: C:\Users\f9104\Desktop\cut\cut\train\Anterior\stream1_20230727_013154_frame0000192_right.jpg
Anterior: C:\Users\f9104\Desktop\cut\cut\train\Anterior\stream1_20220711_085439_frame0000262_left.jpg
Anterior: C:\Users\f9104\Desktop\cut\cut\train\Anterior\stream1_20230920_155516_frame0000367_right.jpg
Anterior: C:\Users\f9104\Desktop\cut\cut\train\Anterior\stream1_20231004_200247_frame0001035_right.jpg
Anterior: C:\Users\f9104\Desktop\cut\cut\train\Anterior\stream1_20230727_013154_frame0000109_left.jpg
Anterior: C:\Users\f9104\Desktop\cut\cut\train\Anterior\stream1_20230815_163628_frame0000477_right.jpg
Posterior: C:\Users\f9104\Desktop\cut\cut\train\Posterior\stream1_20230912_150459_frame0000457_left.jpg
Posterior: C:\Users\f9104\Desktop\cut\cut\train\Posterior\stream1_20220727_174248_frame0000142_left.jpg
Anterior: C:\Users\f9104\Desktop\cut\cut\train\Anterior\stream1_20230912_170544_frame0000725_right.jpg
Anterior: C:\Users\f9104\Desktop\cut\cut\train\Anterior\stream1_20231004_

In [8]:
import os
import random
from sklearn.model_selection import KFold

# 準備圖像路徑和標籤
def load_image_paths_with_labels(base_dirs):
    image_paths = []
    for base_dir in base_dirs:
        label = os.path.basename(base_dir)
        for fname in os.listdir(base_dir):
            if fname.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
                image_paths.append((os.path.join(base_dir, fname), label))
    return image_paths

# 資料夾路徑

all_image_data = load_image_paths_with_labels([anterior_path, posterior_path])

# 打亂資料
random.shuffle(all_image_data)

# 分離圖像路徑與標籤
image_paths = [x[0] for x in all_image_data]
labels = [x[1] for x in all_image_data]

# 5 折交叉驗證
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 產生 5 fold 的訓練集與驗證集
for fold, (train_index, val_index) in enumerate(kf.split(image_paths)):
    print(f"\nFold {fold+1}")
    train_paths = [image_paths[i] for i in train_index]
    val_paths = [image_paths[i] for i in val_index]
    train_labels = [labels[i] for i in train_index]
    val_labels = [labels[i] for i in val_index]

    print(f"訓練集數量: {len(train_paths)}, 驗證集數量: {len(val_paths)}")

    


Fold 1
訓練集數量: 16000, 驗證集數量: 4000

Fold 2
訓練集數量: 16000, 驗證集數量: 4000

Fold 3
訓練集數量: 16000, 驗證集數量: 4000

Fold 4
訓練集數量: 16000, 驗證集數量: 4000

Fold 5
訓練集數量: 16000, 驗證集數量: 4000


參數

In [9]:
import os
import random
from PIL import Image
from sklearn.model_selection import KFold

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader


In [10]:
train_transform = transforms.Compose([
    transforms.Resize((448, 448)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((448, 448)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


model

EarlyStopping

In [11]:
class EarlyStopping:
    def __init__(self, patience=5):
        self.patience = patience
        self.counter = 0
        self.best_acc = 0
        self.early_stop = False

    def __call__(self, acc):
        if acc > self.best_acc:
            self.best_acc = acc
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True


In [12]:
from torch.utils.data import Dataset
from PIL import Image

class ImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = 0 if self.labels[idx] == 'Anterior' else 1
        return img, label


In [13]:
import torch
import torch.backends.cudnn as cudnn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device.type == 'cuda':
    cudnn.benchmark = True    # 對固定輸入尺寸有加速效果
    cudnn.enabled = True      # 開啟 cuDNN 支援（預設為 True）
    print("cuDNN is enabled:", cudnn.enabled)


cuDNN is enabled: True


In [14]:
from torchvision import models
import torch.nn as nn

def get_model():
    model = models.resnet50(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, 2)  # 二分類
    return model.to(device)


In [15]:
output_dirs = []
for fold in range(5):
    fold_dir = f"./fold_{fold + 1}"
    model_dir = os.path.join(fold_dir, "models")
    csv_path = os.path.join(fold_dir, "metrics.csv")

    os.makedirs(model_dir, exist_ok=True)

    output_dirs.append({
        "fold_dir": fold_dir,
        "model_dir": model_dir,
        "csv_path": csv_path
    })


In [17]:
dataset = ImageDataset(image_paths, labels, transform=None)

# 檢查是否有資料
if len(dataset) == 0:
    print("Dataset is empty.")
else:
    print(f"Dataset contains {len(dataset)} items.")


Dataset contains 20000 items.


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("目前使用設備：", device)
if device.type == "cuda":
    print("GPU 名稱：", torch.cuda.get_device_name(0))
    print("cuDNN 是否啟用：", torch.backends.cudnn.enabled)
else:
    print(" 沒有偵測到可用的 GPU，將使用 CPU")


目前使用設備： cuda
GPU 名稱： NVIDIA GeForce RTX 4080 SUPER
cuDNN 是否啟用： True


In [ ]:
from tqdm import tqdm
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import os

# 用來儲存每個 fold 的最佳結果
fold_summaries = []

# 進行 5-fold Cross Validation
for fold, (train_index, val_index) in enumerate(kf.split(image_paths)):
    print(f"\n--- Fold {fold + 1} ---")

    # 切分資料：train / val+test
    train_paths = [image_paths[i] for i in train_index]
    train_labels = [labels[i] for i in train_index]

    val_paths_full = [image_paths[i] for i in val_index]
    val_labels_full = [labels[i] for i in val_index]

    split_idx = len(val_paths_full) // 2
    val_paths = val_paths_full[:split_idx]
    val_labels = val_labels_full[:split_idx]
    test_paths = val_paths_full[split_idx:]
    test_labels = val_labels_full[split_idx:]

    # 建立 Dataset 和 DataLoader
    train_dataset = ImageDataset(train_paths, train_labels, transform=train_transform)
    val_dataset = ImageDataset(val_paths, val_labels, transform=val_transform)
    test_dataset = ImageDataset(test_paths, test_labels, transform=val_transform)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=10)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=10)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=10)

    # 初始化模型與訓練參數
    model = get_model()
    model = models.resnet50(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, 2)  # 二分類
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    early_stopping = EarlyStopping(patience=5)

    metrics = []

    # 訓練過程
    for epoch in range(30):
        model.train()
        total_loss, correct, total = 0, 0, 0

        loop = tqdm(train_loader, desc=f"Fold {fold+1} | Epoch {epoch+1} [Train]", leave=False)
        for images, targets in loop:
            images, targets = images.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == targets).sum().item()
            total += targets.size(0)
            loop.set_postfix(loss=loss.item())

        train_acc = correct / total
        train_loss = total_loss / len(train_loader)

        # 驗證
        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        all_preds = []
        all_targets = []

        with torch.no_grad():
            for images, targets in tqdm(val_loader, desc=f"Fold {fold+1} | Epoch {epoch+1} [Val]", leave=False):
                images, targets = images.to(device), targets.to(device)
                outputs = model(images)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

                preds = outputs.argmax(dim=1)
                val_correct += (preds == targets).sum().item()
                val_total += targets.size(0)

                all_preds.extend(preds.cpu().numpy())
                all_targets.extend(targets.cpu().numpy())

        val_acc = val_correct / val_total
        val_loss /= len(val_loader)

        print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Acc={train_acc:.4f} | Val Loss={val_loss:.4f}, Acc={val_acc:.4f}")

        metrics.append({
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'train_acc': train_acc,
            'val_loss': val_loss,
            'val_acc': val_acc
        })

        if val_acc >= early_stopping.best_acc:
            model_save_path = os.path.join(output_dirs[fold]["model_dir"], "best_model.pth")
            torch.save(model.state_dict(), model_save_path)

        early_stopping(val_acc)
        if early_stopping.early_stop:
            print("Early stopping triggered.")
            break

    # 儲存訓練過程紀錄
    df_metrics = pd.DataFrame(metrics)
    df_metrics.to_csv(output_dirs[fold]["csv_path"], index=False)

    # 混淆矩陣圖
    cm = confusion_matrix(all_targets, all_preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Anterior', 'Posterior'])
    plt.figure(figsize=(5, 5))
    disp.plot(cmap='Blues', values_format='d')
    plt.title(f"Fold {fold+1} Confusion Matrix")
    plt.savefig(os.path.join(output_dirs[fold]["fold_dir"], f"confusion_matrix.png"))
    plt.close()

    # Accuracy 折線圖
    plt.figure()
    plt.plot(df_metrics['epoch'], df_metrics['train_acc'], label='Train Accuracy')
    plt.plot(df_metrics['epoch'], df_metrics['val_acc'], label='Val Accuracy')
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title(f"Fold {fold+1} Accuracy")
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_dirs[fold]["fold_dir"], f"accuracy_plot.png"))
    plt.close()

    # Loss 折線圖
    plt.figure()
    plt.plot(df_metrics['epoch'], df_metrics['train_loss'], label='Train Loss')
    plt.plot(df_metrics['epoch'], df_metrics['val_loss'], label='Val Loss')
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(f"Fold {fold+1} Loss")
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_dirs[fold]["fold_dir"], f"loss_plot.png"))
    plt.close()

    # 測試集準確率計算
    test_correct, test_total = 0, 0
    with torch.no_grad():
        for images, targets in test_loader:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            preds = outputs.argmax(dim=1)
            test_correct += (preds == targets).sum().item()
            test_total += targets.size(0)
    test_acc = test_correct / test_total
    print(f" Fold {fold+1} Test Accuracy: {test_acc:.4f}")

    # 收集最佳結果
    best_row = df_metrics.loc[df_metrics['val_acc'].idxmax()]
    fold_summaries.append({
        'fold': fold + 1,
        'best_epoch': int(best_row['epoch']),
        'train_acc': best_row['train_acc'],
        'val_acc': best_row['val_acc'],
        'train_loss': best_row['train_loss'],
        'val_loss': best_row['val_loss'],
        'test_acc': test_acc
    })

# 輸出 summary.csv
df_summary = pd.DataFrame(fold_summaries)
mean_row = {
    'fold': 'Average',
    'best_epoch': '-',
    'train_acc': df_summary['train_acc'].mean(),
    'val_acc': df_summary['val_acc'].mean(),
    'train_loss': df_summary['train_loss'].mean(),
    'val_loss': df_summary['val_loss'].mean(),
    'test_acc': df_summary['test_acc'].mean()
}
df_summary = pd.concat([df_summary, pd.DataFrame([mean_row])], ignore_index=True)
df_summary.to_csv("summary.csv", index=False)
print("\n summary.csv saved. Here is the summary:")
print(df_summary)




--- Fold 1 ---


C:\Users\f9104\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\f9104\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Fold 1 | Epoch 1 [Train]:   0%|          | 0/500 [00:00<?, ?it/s]